# **The code has no output due to medical confidentiality**

### **Install packages**

In [ ]:
install.packages("sparklyr")
knitr::opts_chunk$set(echo = TRUE)

In [ ]:
# Install packages
install.packages('basecamb')
install.packages("dunn.test")
install.packages("igraph")
install.packages("ggridges")
install.packages("ggplot2")
install.packages("viridis")
install.packages('survminer')
library(igraph)
library(tidyverse)
library(ggplot2)
library(viridis)
library(ggridges)
library(tidymodels)
library(lubridate)
library(dplyr)
library(dunn.test)

In [ ]:
# Install packages
install.packages("gapminder")
library(gapminder)

### **Research**

#### **Functions**

In [ ]:
# A function that adds binary columns for death in immidiate time, 1 year and 5 years after hospitalization, death_date equal to NA will be 0
add_binary_deaths <- function(tablename) {
    tablename <- tablename %>% 
    mutate(death_immidiate = ifelse((as.Date(death_date) - as.Date(cohort_start_date)) > 90 , 0, 1),
           death_after_one_year = ifelse((as.Date(death_date) - as.Date(cohort_start_date)) > 365, 0, 1),
           # Every 4 years an additional day is added
           death_after_five_years = ifelse((as.Date(death_date) - as.Date(cohort_start_date)) > 365*5 + 1, 0, 1),
           
           death_immidiate = ifelse(is.na(death_date), 0, death_immidiate),
           death_after_one_year = ifelse(is.na(death_date), 0, death_after_one_year),
           death_after_five_years = ifelse(is.na(death_date), 0, death_after_five_years))
}

In [ ]:
# A function that returns the logistic model for a specific dependent variable and dependent variables: bmi, gender and age 
logistic_model_from_list <- function(tablename, dependent_variable) {
variables_list <-colnames(tablename)
independent_vars <- paste0(variables_list[4])
    
    for (i in 3: 2)
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    logistic_model <- glm(paste0(dependent_variable, "~", independent_vars), data = tablename, family = binomial)
    #logistic_table <- tidy(logistic_model)
    #logistic_table <- logistic_table %>% mutate(odd_ratio = exp(estimate)) 
    #confidence_interval <- as_tibble(confint(logistic_model))
    #logistic_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
    #logistic_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
    #logistic_table <- logistic_table %>% mutate("estimate+-[]" = estimate - logistic_table$"2.5 %")
    #logistic_table
    basecamb::or_model_summary(logistic_model)
}

In [ ]:
# A function that returns the logistic model for a specific dependent variable and dependent variables: bmi, gender and age 
logistic_model_from_list_with_groups <- function(tablename, dependent_variable) {
    
  tablename <-tablename %>% mutate(bmi_group = as.factor(bmi_group))
  tablename <- tablename %>%
      mutate(bmi_group = relevel(bmi_group, ref = "BMI group 18.5-25"))
  variables_list <-colnames(tablename)
  independent_vars <- paste0(variables_list[4])
    
    for (i in 3: 2)
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    logistic_model <- glm(paste0(dependent_variable, "~", independent_vars), data = tablename, family = binomial)
    logistic_table <- tidy(logistic_model)
    logistic_table <- logistic_table %>% mutate(odd_ratio = exp(estimate)) 
    confidence_interval <- as_tibble(confint(logistic_model))
    logistic_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
    logistic_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
    logistic_table <- logistic_table %>% mutate("estimate+-[]" = estimate - logistic_table$"2.5 %")
    logistic_table
}

In [ ]:
# A function that returns the logistic model for a specific dependent variable and dependent variables: bmi, gende, age, age_greater_65, age_greater_65*bmi
logistic_model_from_list_with_interaction<- function(tablename, dependent_variable) {
    variables_list <-colnames(tablename)
    independent_vars <- paste0("bmi + gender + age + age_greater_65 + age_greater_65*bmi")
    logistic_model <- glm(paste0(dependent_variable, "~", independent_vars), data = tablename, family = binomial)
    logistic_table <- tidy(logistic_model)
    logistic_table <- logistic_table %>% mutate(odd_ratio = exp(estimate)) 
    confidence_interval <- as_tibble(confint(logistic_model))
    logistic_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
    logistic_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
    logistic_table <- logistic_table %>% mutate("estimate+-[]" = estimate - logistic_table$"2.5 %")
    logistic_table
}

In [ ]:
# A function that returns the linear model for a specific dependent variable and dependent variables: bmi, gender and age 
linear_model_from_list <- function(tablename, dependent_variable) {
variables_list <-colnames(tablename)
independent_vars <- paste0(variables_list[4])
    
    for (i in 3: 2)
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    linear_model <- lm(paste0(dependent_variable, "~", independent_vars), data = tablename)
    linear_table <- tidy(linear_model)
    confidence_interval <- as_tibble(confint(linear_model))
    linear_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
    linear_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
    linear_table <- linear_table %>% mutate("estimate+-[]" = estimate - linear_table$"2.5 %")
    linear_table
}

In [ ]:
# A function that gets a dataframe and a death_column (immidiate/after a year/after five years) and returns prop death for each bmi group
prop_death <-function(tablename, column) {
    x <- tablename %>% 
    count(bmi_group, tablename[,column]) %>%
    group_by(bmi_group)%>%
    mutate(death_prop = n / sum(n),
          group_size = sum(n))
        
    colnames(x)[2] <- 'bin_death'
    colnames(x)[3] <- 'death_records'

    x %>% 
    filter(bin_death == 1) %>%
    select(bmi_group,death_prop, death_records,group_size) 
}

In [ ]:
# A function that gets a dataframe and returns average hospitalztion time for each bmi group 
average_hosp_duration <-function(tablename) {
    tablename %>% 
    group_by(bmi_group)%>%
    summarize(mean_hospitalization_duration = mean(days_of_hospitalization, na.rm=TRUE),median = median(days_of_hospitalization, na.rm=TRUE), group_size=n()) 
}

In [ ]:
# A function that gets a dataframe and returns a list that contain death values (immidiate/after one year/after five years) for each bmi group 
find_death_values <- function(tablename)
    {
    # vector that contain all the bmi_groups 
    unique_bmi_group <- sort(c(unique(tablename$bmi_group)))
    death_column <- tablename[c(32)]
    
    # empty vector in size number of other bmi_groups
    values_by_group <- vector("list", length(unique_bmi_group))
    # a pointer for the element's list
    i = 1

    for (x in unique_bmi_group)
    {
        # define 6 different death values by bmi_group  each group values by bmi_group
         y <- (tablename %>%
        filter(bmi_group == x))[[32]] ## <- take the last column (immidiate/after one year/after five years)
        values_by_group[[i]] <- as.numeric(y)
        # points to the next element
        i <- i+1 
        }
    values_by_group
    }


In [ ]:
# A function that returns the logistic model for a specific dependent variable and dependent variables: bmi, gender and age 
coxph_model_from_list <- function(tablename, dependent_variable1, dependent_variable2) {
variables_list <-colnames(tablename)
independent_vars <- paste0(variables_list[5])


    for (i in 4: 3)
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    independent_vars
    coxph_model <- coxph(Surv(tablename[[dependent_variable1]],tablename[[dependent_variable2]]) ~ bmi + gender + age, data = tablename)
    coxph_table <- tidy(coxph_model)
    coxph_table <- coxph_table %>% mutate(odd_ratio = exp(estimate)) 
    confidence_interval <- as_tibble(confint(coxph_model))
    coxph_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
    coxph_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
    coxph_table <- coxph_table %>% mutate("estimate+-[]" = estimate - coxph_table$"2.5 %")
    coxph_table
}

In [ ]:
# A function that returns the logistic model for a specific dependent variable and dependent variables: bmi, gender and age 
coxph_group_model_from_list <- function(tablename, dependent_variable1, dependent_variable2) {
variables_list <-colnames(tablename)
independent_vars <- paste0(variables_list[5])


    for (i in 4: 3)
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    independent_vars
    coxph_model <- coxph(Surv(tablename[[dependent_variable1]],tablename[[dependent_variable2]]) ~ bmi_group + gender + age, data = tablename)
    coxph_table <- tidy(coxph_model)
    coxph_table <- coxph_table %>% mutate(odd_ratio = exp(estimate)) 
    confidence_interval <- as_tibble(confint(coxph_model))
    coxph_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
    coxph_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
    coxph_table <- coxph_table %>% mutate("estimate+-[]" = estimate - coxph_table$"2.5 %")
    coxph_table
}

In [ ]:
# A function that returns a coxph_model for graph
coxph_graph <- function(tablename, dependent_variable1, dependent_variable2) {
variables_list <-colnames(tablename)
independent_vars <- paste0(variables_list[5])
    for (i in 4: 3)
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    independent_vars
    coxph_model <- coxph(Surv(tablename[[dependent_variable1]],tablename[[dependent_variable2]]) ~ bmi + gender + age, data = tablename)
    coxph_model
    }

In [ ]:
# A function that returns a coxph_group_model for graph
coxph_group_graph <- function(tablename, dependent_variable1, dependent_variable2) {
variables_list <-colnames(tablename)
independent_vars <- paste0(variables_list[5])
    for (i in 4: 3)
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    independent_vars
    coxph_model <- coxph(Surv(tablename[[dependent_variable1]],tablename[[dependent_variable2]]) ~ bmi_group + gender + age, data = tablename)
    coxph_model
    }

In [ ]:
# A function that returns the linear model for a specific dependent variable and dependent variables: bmi, gender and age
continuous_confounding_model <- function(tablename, dependent_variable) {
    # get the independent variables
    variables_list <- colnames(tablename)
    variables_list <- variables_list[c(2:32)]
    independent_vars <- paste0(variables_list[1])
    
   for (i in 2: length(variables_list))
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    
    model <- lm(paste0(dependent_variable, "~", independent_vars), data = tablename)
    model_table <- tidy(model)
    model_table <- model_table %>% mutate(odd_ratio = exp(estimate)) 
    confidence_interval <- as_tibble(confint(model))
    model_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
    model_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
    model_table <- model_table %>% mutate("estimate+-[]" = estimate - model_table$"2.5 %")
    model_table %>% arrange(p.value)
}

In [ ]:
# A function that returns the linear model for a specific dependent variable and dependent variables: bmi_group, gender and age

groups_confounding_model <- function(tablename, dependent_variable) {
    # get the independent variables
    tablename <- tablename %>% mutate(bmi_group = as.factor(bmi_group))
    tablename <- tablename %>%
      mutate(bmi_group = relevel(bmi_group, ref = "BMI group 18.5-25"))

    # get the independent variables
    variables_list <- colnames(tablename)
    variables_list <- variables_list[c(2:32)]
    independent_vars <- paste0(variables_list[1])
    
   for (i in 2: length(variables_list))
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    
    model <- lm(paste0(dependent_variable, "~", independent_vars), data = tablename)
    model_table <- tidy(model)
    model_table <- model_table %>% mutate(odd_ratio = exp(estimate)) 
    confidence_interval <- as_tibble(confint(model))
    model_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
    model_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
    model_table <- model_table %>% mutate("estimate+-[]" = estimate - model_table$"2.5 %")
    model_table %>% arrange(p.value)
}

In [ ]:
# A function that gets dunn test variable and returns plot that show the connection between similiar groups by the dunn test
get_similar_groups <-function(dunn_test) {
    list_similar_pairs <- c() 
    list_p_adj <- dunn_test$P.adjusted
    for (x in 1:15){
        if (list_p_adj[x] > 0.025){
            list_similar_pairs <- append(list_similar_pairs,stringr::str_extract_all(dunn_test$comparisons[x], "\\d+"))
            }
        }
    # create a df of pairs simmilar vertices
    df <- data.frame(P1 = c('15-18.5','18.5-25','25-30','30-35','35-40','40-45'), P2 = c('15-18.5','18.5-25','25-30','30-35','35-40','40-45')) 
    for (pair in list_similar_pairs){
        
        p1 <- as.numeric(unlist(pair[1]))
        p2 <- as.numeric(unlist(pair[2]))
        df[nrow(df) +1,] <- c(p1, p2)
            }
    df[df == 1] <- '15-18.5'  
    df[df == 2] <- '18.5-25'
    df[df == 3] <- '25-30'
    df[df == 4] <- '30-35'
    df[df == 5] <- '35-40'
    df[df == 6] <- '40-45'
    
    g_1 <- graph.data.frame(df,directed=F)
    plot(g_1,  vertex.size=30)
}

In [ ]:
# A function that gets dataframe and returns a list that contain death values (immidiate/after one year/after five years) for each bmi group 
find_death_values <- function(tablename)
    {
    # vector that contain all the bmi_groups 
    unique_bmi_group <- sort(c(unique(tablename$bmi_group)))
    death_column <- tablename[c(1)]
    
    # empty vector in size number of other bmi_groups
    values_by_group <- vector("list", length(unique_bmi_group))
    # a pointer for the element's list
    i = 1
    for (x in unique_bmi_group)
    {
        # define 6 different death values by bmi_group  each group values by bmi_group
         y <- (tablename %>%
        filter(bmi_group == x))[[1]] ## <- take the last column (immidiate/after one year/after five years)
        
        values_by_group[[i]] <- as.numeric(y)
        # points to the next element
        i <- i+1 
        }
    values_by_group
    }


### **Main research question - what is the association between BMI and mortality for different time horizons:**

In [ ]:
obesity <- readRDS("obesity.RDS")

##### **Add binary death columns**

In [ ]:
# Add binary death columns for immidiate time, one year and five years
obesity_binary <- add_binary_deaths(obesity)

##### **death ~ bmi + gender + age**

##### **1. Immidiate time**

In [ ]:
# Select only columns with binary_death, age, gender and bmi
obesity_binary_imm <- obesity_binary[c(37,2:3,8)]

# logistic regression for immidiate time period
imm_res <- logistic_model_from_list(obesity_binary_imm,  "death_immidiate")

In [ ]:
imm_res

##### **2. After 1 year**

In [ ]:
# Select only columns with binary_death, age, gender and bmi
# there were no cohort_start_date after 2021-07-02, thus there is no need to discard any rows
obesity_binary_one_year <- obesity_binary[c(38,2:3,8)]

# logistic regression for one year period
one_year_res <- logistic_model_from_list(obesity_binary_one_year,  "death_after_one_year")

In [ ]:
one_year_res

##### **3. After 5 years**

In [ ]:
# Select only columns with binary_death, age, gender and bmi
# 2017-07-02 is the latest cohort_end_date
not_include_5_years = obesity_binary %>% filter(as.Date(cohort_start_date) > '2017-07-02') %>% select(person_id)
obesity_binary_five_years <- obesity_binary %>% filter(!(person_id %in% not_include_5_years$person_id))
obesity_binary_five_years <- obesity_binary_five_years[c(39,2:3,8)]

# logistic regression for five years period
five_years_res <- logistic_model_from_list(obesity_binary_five_years,  "death_after_five_years")

In [ ]:
five_years_res

##### **Graph with the 3 models - The mortality rate as a function of BMI, divided by time frame**

In [ ]:
# Create a graph for 3 time frames

new_bmis1 <- seq(15,45)

# Immidiate time coefs
a1 = imm_res$odd_ratio[1]
b1 = imm_res$odd_ratio[2]

# One year coefs
a2 = one_year_res$odd_ratio[1]
b2 = one_year_res$odd_ratio[2]

# Five years coefs
a3 = five_years_res$odd_ratio[1]
b3 = five_years_res$odd_ratio[2]

imm_curve <- 1*b1**(new_bmis1-18.5) 

one_year_curve <- 1*b2**(new_bmis1-18.5)

five_years_curve <- 1*b3**(new_bmis1-18.5) 

a_probs <- imm_curve
b_probs <- one_year_curve
c_probs <- five_years_curve

In [ ]:
# Mortality probability by time frame
plot.data <- data.frame("Immidiate time"=a_probs, "1 year"=b_probs, "5 years" =c_probs, X1=new_bmis1)

plot.data <- rename(plot.data, "Immidiate time" = Immidiate.time, "1 year" =X1.year,"5 years" = X5.years)

plot.data <- gather(plot.data, key=Group, value= prob,"Immidiate time",  "1 year", "5 years")

cols <- c("Immidiate time" = "indianred1",  "1 year" = "green3", "5 years" = "deepskyblue2")

options(repr.plot.width = 10, repr.plot.height = 7)
ggplot(plot.data, aes(x=X1, y=prob, color=Group)) + # asking it to set the color by the variable "group" is what makes it draw three different lines
  geom_line(lwd=2) + theme_classic() +  
  scale_color_manual(values = cols, breaks = c("Immidiate time", "1 year", "5 years"),)  + scale_x_continuous(limits = c(15, 45), breaks = seq(15, 45, by = 5)) +
      labs(x = expression("BMI (kg/m"^"2"*")"), y="Odds ratio", title="How BMI affects hospitalized mortality rate",
       subtitle = "Divided by different time periods") +
        theme(plot.title = element_text(face="bold",size = 14), axis.title.x = element_text(hjust=0.5,size = 14),axis.title.y = element_text(hjust=0.5,size = 14),
              plot.subtitle = element_text(, size = 13), legend.title = element_text(face="bold", size=13), legend.text = element_text(size=13))


In [ ]:
#########################################################################################

##### **death ~ bmi_group + gender + age**

##### **1. Immidiate time**

In [ ]:
# Select only columns with binary_death, age, gende and bmi group
obesity_binary_imm <- obesity_binary[c(37,2:3,6)]

In [ ]:
logistic_model_from_list_with_groups(obesity_binary_imm,  "death_immidiate")

In [ ]:
# Use Kruskal Wallis test
kruskal.test(obesity_binary_imm$death_immidiate ~ bmi_group,
                    data = obesity_binary_imm)

In [ ]:
# Get list that contain death_immidiate values for each bmi group 
obesity_immidiate_values <- find_death_values(obesity_binary_imm)

# Perform Dunn's Test with bonferroni correction for death_immidiate values
test <- dunn.test(x=obesity_immidiate_values, method="bonferroni")

In [ ]:
# Get plot of groups connections
get_similar_groups(test)

In [ ]:
prop_death(obesity_binary_imm, "death_immidiate")

##### **2. After 1 year**

In [ ]:
# Select only columns with binary_death, age, gender and bmi group
obesity_binary_one_year <- obesity_binary[c(38,2:3,6)]

In [ ]:
logistic_model_from_list_with_groups(obesity_binary_one_year,  "death_after_one_year")

In [ ]:
# Use Kruskal Wallis test
kruskal.test(obesity_binary_one_year$death_after_one_year ~ bmi_group,
                    data = obesity_binary_one_year)

In [ ]:
# Get list that contain death_immidiate values for each bmi group 
obesity_one_year_values <- find_death_values(obesity_binary_one_year)

# Perform Dunn's Test with bonferroni correction for death_immidiate values
test <- dunn.test(x=obesity_one_year_values, method="bonferroni")

In [ ]:
# Get plot of groups connections
get_similar_groups(test)

In [ ]:
prop_death(obesity_binary_one_year,  "death_after_one_year")

##### **3. After 5 years**

In [ ]:
# Select only columns with binary_death, age, gender and bmi
# 2017-07-02 is the latest cohort_end_date
not_include_5_years = obesity_binary %>% filter(as.Date(cohort_start_date) > '2017-07-02') %>% select(person_id)
obesity_binary_five_years <- obesity_binary %>% filter(!(person_id %in% not_include_5_years$person_id))
obesity_binary_five_years <- obesity_binary_five_years[c(39,2:3,6)]

In [ ]:
logistic_model_from_list_with_groups(obesity_binary_five_years,  "death_after_five_years")

In [ ]:
# Use Kruskal Wallis test
kruskal.test(obesity_binary_five_years$death_after_five_years ~ bmi_group,
                    data = obesity_binary_five_years)

In [ ]:
# Get list that contain death_immidiate values for each bmi group 
obesity_one_year_values <- find_death_values(obesity_binary_five_years)

# Perform Dunn's Test with bonferroni correction for death_immidiate values
test <- dunn.test(x=obesity_one_year_values, method="bonferroni")

In [ ]:
# Get plot of groups connections
get_similar_groups(test)

In [ ]:
prop_death(obesity_binary_five_years,  "death_after_five_years")

##### **Plot the Odds ratio by period and BMI group**

In [ ]:
full_odd <-  imm_res %>% select('adj_OR')
full_odd <- merge(full_odd, one_year_res[ , c("adj_OR"), drop=FALSE], by="row.names", all=TRUE)
rownames(full_odd) <- full_odd$Row.names
full_odd$Row.names <- NULL
full_odd <- merge(full_odd, five_year_res[ , "adj_OR", drop=FALSE], by="row.names", all=TRUE)
colnames(full_odd)[2:4] <- c('imm_OR', 'one_year_OR', 'five_years_OR')
full_odd <- rbind(full_odd, c('bmi_groupBMI group 18.5-25', 1.,1.,1.))
full_odd <- full_odd[grep('^bmi_group', full_odd$Row.names),]
full_odd$Row.names <- gsub('bmi_groupBMI group ', '', full_odd$Row.names)
colnames(full_odd) <- c('BMI_Group', "Immidiate time", "1 year", "5 years")
full_odd

In [ ]:
full_odd <- full_odd %>% pivot_longer(!BMI_Group, names_to = 'period', values_to='Odds Ratio')
full_odd$period <- as.factor(full_odd$period)
full_odd$BMI_Group <- as.factor(full_odd$BMI_Group)
full_odd$`Odds Ratio` <- as.numeric(full_odd$`Odds Ratio`)
head(full_odd)

In [ ]:
ggplot(full_odd, aes(fill=period, y=`Odds Ratio`, x=BMI_Group)) + geom_bar(position="dodge", stat="identity") +
labs(x = "BMI groups", y="Odds ratio", title="How BMI groups affects hospitalized mortality rate",
       subtitle = "Divided by different time periods") +
scale_y_continuous(limits = c(0, 3), breaks = seq(0, 3, by = 0.5)) +
facet_grid(~fct_relevel(period,"Immidiate time","1 year","5 years")) +
theme_bw() +
theme(axis.text = element_text(size = 11),
      axis.text.x = element_text(angle = 30, vjust = 0.5, hjust=1),
      axis.title.y = element_text(hjust=0.5,size = 14),
      legend.title = element_text(face="bold", size=13),
      legend.text = element_text(size=13),
      plot.title = element_text(face="bold",size = 14), 
      plot.subtitle = element_text(, size = 13), 
      strip.text.x = element_text(size = 13)) # Facet titles

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 7)
ggplot(full_odd, aes(x=BMI_Group, group = period, colour = period, y=`Odds Ratio`)) + 
    geom_line() +
    theme_linedraw() +
    labs(x = "BMI groups", y="Odds ratio", title="How BMI groups affects hospitalized mortality rate",
    subtitle = "Divided by different time periods") +
    theme(plot.title = element_text(face="bold",size = 14), 
          axis.title.x = element_text(hjust=0.5,size = 14),
          axis.title.y = element_text(hjust=0.5,size = 14),
          plot.subtitle = element_text(, size = 13), 
          legend.title = element_text(face="bold", size=13),
          panel.grid = element_line(
                                  linewidth = 0.9,
                                  linetype = 2),
          axis.text = element_text(size = 11),
          axis.text.x=element_text(angle=30,hjust=1)) + 
        scale_y_continuous(limits = c(0, 2.5), breaks = seq(0, 2.5, by = 0.5)) +
        annotate(geom="text", x= 0.7 , y=2.3, label="Immidiate", color="indianred1",size = 4.5) +
        annotate(geom="text", x= 0.7 , y=2.5, label="One year", color="green3",size = 4.5) + 
        annotate(geom="text", x= 0.7 , y=2.1, label="Five years", color="deepskyblue2",size = 4.5)

##### **death ~ bmi + gender + age + age_greater_65 + age_greater_65*bmi**

##### **1. Immidiate time**

In [ ]:
# Select only columns with age, gender and bmi 
obesity_binary_imm <- obesity_binary[c(37,2:3,8)]
obesity_binary_imm <- obesity_binary_imm %>% mutate(age_greater_65 = ifelse(age >= 65 , 1, 0))

In [ ]:
logistic_model_from_list_with_interaction(obesity_binary_imm,  "death_immidiate")

##### **2. After 1 year**

In [ ]:
# Select only columns with age, gender and bmi 
obesity_binary_one_year <- obesity_binary[c(38,2:3,8)]
obesity_binary_one_year <- obesity_binary_one_year %>% mutate(age_greater_65 = ifelse(age >= 65 , 1, 0))

In [ ]:
logistic_model_from_list_with_interaction(obesity_binary_one_year,  "death_after_one_year")

##### **3. After 5 years**

In [ ]:
# Select only columns with age, gender and bmi 
# 2017-07-02 is the latest cohort_end_date
not_include_5_years = obesity_binary %>% filter(as.Date(cohort_start_date) > '2017-07-02') %>% select(person_id)
obesity_binary_five_years <- obesity_binary %>% filter(!(person_id %in% not_include_5_years$person_id))
obesity_binary_five_years <- obesity_binary_five_years[c(39,2:3,8)]
obesity_binary_five_years <- obesity_binary_five_years %>% mutate(age_greater_65 = ifelse(age >= 65 , 1, 0))

In [ ]:
logistic_model_from_list_with_interaction(obesity_binary_five_years,  "death_after_five_years")

In [ ]:
##########################################

#### **Survival analysis**

In [ ]:
library("survival")

In [ ]:
library("survminer")

In [ ]:
head(obesity, 5)

**Time_to_event (in days)= min(cohort_end_date, death_date, cohort_start_date + X years) - cohort_start_date**

**Types of events:**

1. Discontinuation of patient follow-up
2. Death of the patient
3. A patient has not (yet) experienced the event of interest

In [ ]:
obesity_binary <- add_binary_deaths(obesity)

# Finding the time when a person dies/ no longer participates in the study/ not in the time frame
obesity_binary <- obesity_binary %>% rowwise() %>% mutate(time_to_event_imm = ifelse(!is.na(death_date), min(as.Date(cohort_end_date), as.Date(death_date) ,as.Date(cohort_start_date) + 90) - cohort_start_date,
                                                                                min(as.Date(cohort_end_date),as.Date(cohort_start_date) + 90) - cohort_start_date),
                                                         time_to_event_one_year = ifelse(!is.na(death_date), min(as.Date(cohort_end_date), as.Date(death_date) ,as.Date(cohort_start_date) + 365) - cohort_start_date,
                                                                                min(as.Date(cohort_end_date),as.Date(cohort_start_date) + 365) - cohort_start_date),
                                                         time_to_event_five_years = ifelse(!is.na(death_date), min(as.Date(cohort_end_date), as.Date(death_date) ,as.Date(cohort_start_date) + 5*365 + 1) - cohort_start_date,
                                                                                min(as.Date(cohort_end_date),as.Date(cohort_start_date) + 5*365 + 1) - cohort_start_date))

#### Continuous BMI

##### **1. Immidiate time**

In [ ]:
# Select only columns with binary_death, time_to_event, bmi, age, gender
obesity_binary_imm <- obesity_binary[c(37,40,2:3,8)]

In [ ]:
# Survival analisys model for immidiate time period
coxph_model_from_list(obesity_binary_imm,"time_to_event_imm", "death_immidiate")

##### **2. After 1 year**

In [ ]:
# Select only columns with binary_death, time_to_event, bmi, age, gender
obesity_binary_one_year <- obesity_binary[c(38,41,2:3,8)]

In [ ]:
# Survival analisys model for one year period
coxph_model_from_list(obesity_binary_one_year,"time_to_event_one_year", "death_after_one_year")

##### **3. After 5 years**

In [ ]:
# Select only columns with binary_death, time_to_event, bmi, age, gender
obesity_binary_five_years <- obesity_binary[c(39,42,2:3,8)]

In [ ]:
# Survival analisys model for five years period
coxph_model_from_list(obesity_binary_five_years,"time_to_event_five_years", "death_after_five_years")

In [ ]:
# Plot the baseline survival function
five_years_res <- coxph_graph(obesity_binary_five_years,"time_to_event_five_years", "death_after_five_years")
ggsurvplot(survfit(five_years_res), legend = "none", xlab = "Time from hospitalization (days)", font.x = c(14),font.y = c(14),
           palette = '#2E9FDF',
           ggtheme = theme_minimal(), data = obesity_binary_five_years)

In [ ]:
#################################################

#### BMI groups

##### **1. Immidiate time**

In [ ]:
# Select only columns with binary_death, time_to_event, bmi_groups, age, gender
obesity_binary_imm <- obesity_binary[c(37,40,2:3,6)]

obesity_binary_imm <- obesity_binary_imm %>% mutate(bmi_group = as.factor(bmi_group))
obesity_binary_imm <- obesity_binary_imm %>%
      mutate(bmi_group = relevel(bmi_group, ref = "BMI group 18.5-25"))

In [ ]:
# Survival analisys groups model for immidiate time period
coxph_group_model_from_list(obesity_binary_imm,"time_to_event_imm", "death_immidiate") %>% arrange(term)

In [ ]:
obesity_binary_imm <- obesity_binary[c(37,40,2:3,6)]
prop_death(obesity_binary_imm, "death_immidiate") 

In [ ]:
# Plot the baseline survival function
options(repr.plot.width = 10, repr.plot.height = 7)
fit <- survfit(Surv(time_to_event_imm, death_immidiate) ~ bmi_group, data = obesity_binary_imm )
ggsurvplot(fit,title = "Survival probabilty by time from hospitalization", subtitle = "Immidiate time period: 90 days", legend = "right", legend.labs = c("15-18.5","18.5-25","25-30", "30-35", "35-40","40-45"),legend.title = "BMI group", xlab = "Time from hospitalization (days)", font.x = c(14),font.y = c(14),font.legend = c(13),font.title = c(14,"bold"),font.subtitle =c(13),ylim = c(0.7,1),
          risk.table.col = "strata", # Change risk table color by groups
          ggtheme = theme_bw(), # Change ggplot2 theme
          xlim = c(0, 100)) +
  guides(color = guide_legend(reverse=TRUE))

##### **2. After 1 year**

In [ ]:
# Select only columns with binary_death, time_to_event, bmi_groups, age, gender
obesity_binary_one_year <- obesity_binary[c(38,41,2:3,6)]
obesity_binary_one_year <- obesity_binary_one_year %>% mutate(bmi_group = as.factor(bmi_group))
obesity_binary_one_year <- obesity_binary_one_year %>%
      mutate(bmi_group = relevel(bmi_group, ref = "BMI group 18.5-25"))

In [ ]:
# Survival analisys groups model for one year period
coxph_group_model_from_list(obesity_binary_one_year,"time_to_event_one_year", "death_after_one_year") %>% arrange(term)

In [ ]:
obesity_binary_one_year <- obesity_binary[c(38,41,2:3,6)]
prop_death(obesity_binary_one_year, "death_after_one_year") 

In [ ]:
# Plot the baseline survival function
options(repr.plot.width = 10, repr.plot.height = 7)
fit <- survfit(Surv(time_to_event_one_year, death_after_one_year) ~ bmi_group, data = obesity_binary_one_year )
ggsurvplot(fit,title = "Survival probabilty by time from hospitalization", subtitle = "One year period", legend = "right", legend.labs = c("15-18.5","18.5-25","25-30", "30-35", "35-40","40-45"),legend.title = "BMI group", xlab = "Time from hospitalization (days)", font.x = c(14),font.y = c(14),font.legend = c(13),font.title = c(14,"bold"),font.subtitle =c(13),ylim = c(0.5,1),
          risk.table.col = "strata", # Change risk table color by groups
          ggtheme = theme_bw(), # Change ggplot2 theme
          xlim = c(0, 400)) +
  guides(color = guide_legend(reverse=TRUE))

##### **3. After 5 years**

In [ ]:
# Select only columns with binary_death, time_to_event, bmi_groups, age, gender
obesity_binary_five_years <- obesity_binary[c(39,42,2:3,6)]

obesity_binary_five_years <- obesity_binary_five_years %>% mutate(bmi_group = as.factor(bmi_group))
obesity_binary_five_years <- obesity_binary_five_years %>%
      mutate(bmi_group = relevel(bmi_group, ref = "BMI group 18.5-25"))

In [ ]:
# Survival analisys groups model for 5 years period
coxph_group_model_from_list(obesity_binary_five_years,"time_to_event_five_years", "death_after_five_years") %>% arrange(term)

In [ ]:
obesity_binary_five_years <- obesity_binary[c(39,42,2:3,6)]
prop_death(obesity_binary_five_years, "death_after_five_years") 

In [ ]:
# Plot the baseline survival function
options(repr.plot.width = 10, repr.plot.height = 7)
fit <- survfit(Surv(time_to_event_five_years, death_after_five_years) ~ bmi_group, data = obesity_binary_five_years )
ggsurvplot(fit,title = "Survival probabilty by time from hospitalization", subtitle = "Five years period", legend = "right", legend.labs = c("15-18.5","18.5-25","25-30", "30-35", "35-40","40-45"),legend.title = "BMI group", xlab = "Time from hospitalization (days)", font.x = c(14),font.y = c(14),font.legend = c(13),font.title = c(14,"bold"),font.subtitle =c(13),ylim = c(0.25,1),
          risk.table.col = "strata", # Change risk table color by groups
          ggtheme = theme_bw(), # Change ggplot2 theme
          xlim = c(0, 5*365 + 1)) +
  guides(color = guide_legend(reverse=TRUE))

In [ ]:
#################################################

### **Second research question - what is the association of BMI and additional hospitalization outcomes:**

#### **Duration of hospitalizaion**

In [ ]:
# Read obesity_duration
obesity_duration <- readRDS("obesity_duration.RDS")

In [ ]:
# Select only columns with days_of_hospitalization, age, gender and bmi
obesity_duration_bmi <- obesity_duration[c(37,2:3,8)]
obesity_duration_bmi <- obesity_duration_bmi %>% filter(!(is.na(days_of_hospitalization)))

In [ ]:
# Select only columns with days_of_hospitalization, age, gender and bmi groups
obesity_duration_bmi_group <- obesity_duration[c(37,2:3,6)]
obesity_duration_bmi_group <- obesity_duration_bmi_group %>% filter(!(is.na(days_of_hospitalization)))
obesity_duration_bmi_group <- obesity_duration_bmi_group %>% mutate(bmi_group = as.factor(bmi_group))
    obesity_duration_bmi_group <- obesity_duration_bmi_group %>%
      mutate(bmi_group = relevel(bmi_group, ref = "BMI group 18.5-25"))

##### **Get the linear model**

In [ ]:
linear_model_from_list(obesity_duration_bmi,  "days_of_hospitalization")

In [ ]:
linear_model_from_list(obesity_duration_bmi_group,  "days_of_hospitalization")

In [ ]:
# Get list that contain death_immidiate values for each bmi group 
obesity_dur_kw <- find_death_values(obesity_duration_bmi_group)

# Perform Dunn's Test with bonferroni correction for death_immidiate values
test <- dunn.test(x=obesity_dur_kw, method="bonferroni")

In [ ]:
# Get plot of groups connections
get_similar_groups(test)

##### **Change to cumulatative graph, y - precentage of patients until that day**

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 10)
df <- obesity_duration_bmi_group %>% filter(days_of_hospitalization > 0)
ggplot(df, aes(days_of_hospitalization, colour = bmi_group)) + stat_ecdf(size = 1)+
    labs(title = "Cumulative percentage of patients according to the duration of hospitalization", 
       x = "Duration of hospitalization (days)",
       y = "Cumulative percentage of patients",
       subtitle = "Divided by BMI group",
        fill = 'BMI group',
        color = 'BMI group') +
   theme(plot.title = element_text(face="bold",size = 14), axis.title.x = element_text(hjust=0.5,size = 14),axis.title.y = element_text(hjust=0.5,size = 14),
          plot.caption = element_text(hjust=0, size = 14),
              plot.subtitle = element_text(, size = 13), legend.title = element_text(face="bold",size=13), legend.text = element_text(size=13))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 6)
df <- obesity_duration_bmi_group %>% filter(days_of_hospitalization > 0)
ggplot(df, aes(days_of_hospitalization, color = bmi_group)) + stat_ecdf(size = 1.5)+ 
    labs(title = "Cumulative percentage of patients according to the duration of hospitalization", 
       x = "Duration of hospitalization (days)",
       y = "Cumulative percentage of patients",
       subtitle = "Divided by BMI group",
        fill = 'BMI group',
        color = 'BMI group') +
   theme(plot.title = element_text(face="bold",size = 14), axis.title.x = element_text(hjust=0.5,size = 14),axis.title.y = element_text(hjust=0.5,size = 14),
          plot.caption = element_text(hjust=0, size = 14),
              plot.subtitle = element_text(, size = 13), legend.title = element_text(face="bold",size=13), legend.text = element_text(size=13)) +
  coord_cartesian(ylim = c(0.9, 1))

In [ ]:
average_hosp_duration(obesity_duration_bmi_group)

In [ ]:
 # Filter days of hospitalization between 0 to 30
obesity_duration_bmi_groups_for_graph <- obesity_duration[c(37,2:3,6)]
obesity_duration_bmi_groups_for_graph <- obesity_duration_bmi_groups_for_graph %>% 
                                filter(days_of_hospitalization >=0, 
                                      days_of_hospitalization <= 30) %>% mutate(bmi_group = replace(bmi_group, bmi_group == "BMI group 15-18.5", "15-18.5"),
                                                                               bmi_group = replace(bmi_group, bmi_group == "BMI group 18.5-25", "18.5-25"),
                                                                               bmi_group = replace(bmi_group, bmi_group == "BMI group 25-30", "25-30"),
                                                                               bmi_group = replace(bmi_group, bmi_group == "BMI group 30-35", "30-35"),
                                                                               bmi_group = replace(bmi_group, bmi_group == "BMI group 35-40", "35-40"),
                                                                               bmi_group = replace(bmi_group, bmi_group == "BMI group 40-45", "40-45"))

plot_obesity_duration <- ggplot(obesity_duration_bmi_groups_for_graph, aes(x = days_of_hospitalization, y = bmi_group, fill = bmi_group, color = bmi_group))+ 
geom_density_ridges(alpha = 0.5) +
scale_fill_viridis(discrete=TRUE, limits =  c("40-45", "35-40", "30-35","25-30","18.5-25","15-18.5"),direction = -1,end = 0.91)+
scale_color_viridis(discrete=TRUE, limits =  c("40-45", "35-40", "30-35","25-30","18.5-25","15-18.5"),direction=-1,end = 0.91) +
theme_ridges() +
    labs(title = "Hospitalization duration distribution", 
       x = "Duration of hospitalization (days)",
       y = " BMI Group",
       subtitle = "Divided by BMI group",
        fill = 'BMI group',
        color = 'BMI group') +
   theme(plot.title = element_text(face="bold",size = 14), axis.title.x = element_text(hjust=0.5,size = 14),axis.title.y = element_text(hjust=0.5,size = 14),
          plot.caption = element_text(hjust=0, size = 14),
              plot.subtitle = element_text(, size = 13), legend.title = element_text(face="bold",size=13), legend.text = element_text(size=13)) 

plot_obesity_duration 

#### **Re-hospitalizations**

In [ ]:
obesity_re_hospitalizations <- readRDS("obesity_re_hospitalizations.RDS")

In [ ]:
re_hospitalizations_bmi <- obesity_re_hospitalizations[c(37,2:3,8)]

In [ ]:
re_hospitalizations_bmi_group <- obesity_re_hospitalizations[c(37,2:3,6)]
re_hospitalizations_bmi_group <- re_hospitalizations_bmi_group %>% mutate(bmi_group = as.factor(bmi_group))
    re_hospitalizations_bmi_group <- re_hospitalizations_bmi_group %>%
      mutate(bmi_group = relevel(bmi_group, ref = "BMI group 18.5-25"))

##### **Get the logistic model**

In [ ]:
logistic_model_from_list(re_hospitalizations_bmi,  "binary_re_hospitalization")

In [ ]:
logistic_model_from_list(re_hospitalizations_bmi_group,  "binary_re_hospitalization")

In [ ]:
#########################################################################################

### **In addition - What are the confounding variables?**

In [ ]:
library(MASS)

In [ ]:
logistic_model_step <- function(tablename, dependent_variable) {

variables_list <- colnames(tablename)
variables_list <- variables_list[c(2:32)]
independent_vars <- paste0(variables_list[1])

for (i in 2: length(variables_list))
{
   independent_vars <- paste0(independent_vars,"+",variables_list[i])
}
    
logistic_model <- glm(paste0(dependent_variable, "~", independent_vars), 
                      data = tablename, family = "binomial")    
    
step.model <- logistic_model %>% stepAIC(trace = FALSE)  
logistic_table <- tidy(step.model)

confidence_interval <- as_tibble(confint(step.model))
logistic_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
logistic_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
logistic_table <- logistic_table %>% mutate("estimate+-[]" = estimate - logistic_table$"2.5 %")
logistic_table <- logistic_table %>%  arrange(p.value)
logistic_table    
}

In [ ]:
logistic_model_step_groups <- function(tablename, dependent_variable) {
    # get the independent variables
    tablename <- tablename %>% mutate(bmi_group = as.factor(bmi_group))
    tablename <- tablename %>%
      mutate(bmi_group = relevel(bmi_group, ref = "BMI group 18.5-25"))
    # get the independent variables
    variables_list <- colnames(tablename)
    variables_list <- variables_list[c(2:32)]
    independent_vars <- paste0(variables_list[1])
   for (i in 2: length(variables_list))
        {
           independent_vars <- paste0(independent_vars,"+",variables_list[i])
        }
    
    logistic_model <- glm(paste0(dependent_variable, "~", independent_vars), 
                      data = tablename, family = "binomial")    
    
    step.model <- logistic_model %>% stepAIC(trace = FALSE)  
    logistic_table <- tidy(step.model)

    confidence_interval <- as_tibble(confint(step.model))
    logistic_table$"2.5 %" <-  as.numeric(paste(confidence_interval$"2.5 %"))
    logistic_table$"97.5 %" <-  as.numeric(paste(confidence_interval$"97.5 %"))
    logistic_table <- logistic_table %>% mutate("estimate+-[]" = estimate - logistic_table$"2.5 %")
    logistic_table <- logistic_table %>%  arrange(p.value)
    logistic_table    
}

##### Continuous BMI

##### **1. Immidiate time**

In [ ]:
obesity_binary_imm <- obesity_binary[c(37,2:3,8:36)]

In [ ]:
logistic_model_step(obesity_binary_imm, "death_immidiate")

##### **2. After 1 year**

In [ ]:
obesity_binary_one_year <- obesity_binary[c(38,2:3,8:36)]

logistic_model_step(obesity_binary_one_year, "death_after_one_year")

##### **3. After 5 years**

In [ ]:
# 2017-07-02 is the latest cohort_end_date
not_include_5_years = obesity_binary %>% filter(as.Date(cohort_start_date) > '2017-07-02') 

obesity_binary_five_years <- obesity_binary %>% filter(!(person_id %in% not_include_5_years$person_id))
obesity_binary_five_years <- obesity_binary[c(39,2:3,8:36)]

logistic_model_step(obesity_binary_five_years, "death_after_five_years")

##### BMI groups

##### **1. Immidiate time**

In [ ]:
obesity_binary_imm <- obesity_binary[c(37,2:3,6,9:36)]
logistic_model_step_groups(obesity_binary_imm, "death_immidiate")

##### **2. After 1 year**

In [ ]:
obesity_binary_one_year <- obesity_binary[c(38,2:3,6,9:36)]
logistic_model_step_groups(obesity_binary_one_year, "death_after_one_year")

##### **3. After 5 years**

In [ ]:
# 2017-07-02 is the latest cohort_end_date
not_include_5_years = obesity_binary %>% filter(as.Date(cohort_start_date) > '2017-07-02') 
obesity_binary_five_years <- obesity_binary %>% filter(!(person_id %in% not_include_5_years$person_id))
obesity_binary_five_years <- obesity_binary[c(39,2:3,6,9:36)]

logistic_model_step_groups(obesity_binary_five_years, "death_after_five_years")

#### **Extra - when does the obesity paradox disappear?**

In [ ]:
 # A function that adds binary columns from hospitalization to death for x years after hospitalization, death_date equal to NA will be 0
add_binary_death_diff_time <- function(tablename, years) {
     tablename <- tablename %>% 
    mutate(
           # Every 4 years an additional day is added
           death_after_x_years = ifelse((as.Date(death_date) - as.Date(cohort_start_date)) > 365*years + 1 *floor(years/4), 0, 1),
           death_after_x_years = ifelse(is.na(death_date), 0, death_after_x_years))
}

In [ ]:
 # A function that adds time_to_event_x_years column for survival analysis by finding the minimal time when a person dies/ no longer participates in the study/ not in the time frame
add_time_to_event <- function(tablename, years) {
tablename <- tablename %>% rowwise() %>% mutate(time_to_event_x_years =  ifelse(!is.na(death_date), min(as.Date(cohort_end_date), as.Date(death_date) ,as.Date(cohort_start_date) + 365*years + 1 *floor(years/4)) - cohort_start_date,
                                                                                           min(as.Date(cohort_end_date),as.Date(cohort_start_date) + 365*years + 1 *(floor(years/4))) - cohort_start_date))
}

In [ ]:
library("survival")

In [ ]:
obesity <- readRDS("obesity.RDS")

In [ ]:
obesity_binary <- add_binary_deaths(obesity)

# Finding the time when a person dies/ no longer participates in the study/ not in the time frame
obesity_binary <- obesity_binary %>% rowwise() %>% mutate(time_to_event_imm = ifelse(!is.na(death_date), min(as.Date(cohort_end_date), as.Date(death_date) ,as.Date(cohort_start_date) + 90) - cohort_start_date,
                                                                                min(as.Date(cohort_end_date),as.Date(cohort_start_date) + 90) - cohort_start_date),
                                                         time_to_event_one_year = ifelse(!is.na(death_date), min(as.Date(cohort_end_date), as.Date(death_date) ,as.Date(cohort_start_date) + 365) - cohort_start_date,
                                                                                min(as.Date(cohort_end_date),as.Date(cohort_start_date) + 365) - cohort_start_date),
                                                         time_to_event_five_years = ifelse(!is.na(death_date), min(as.Date(cohort_end_date), as.Date(death_date) ,as.Date(cohort_start_date) + 5*365 + 1) - cohort_start_date,
                                                                                min(as.Date(cohort_end_date),as.Date(cohort_start_date) + 5*365 + 1) - cohort_start_date))

# Select only columns with age, gender, bmi, binary_death and time_to_event
obesity_binary_imm <- obesity_binary[c(37,40,2:3,8)]

res <- coxph_model_from_list(obesity_binary_imm,"time_to_event_imm", "death_immidiate")
res <- res[c(1:5)]

paradox_by_year = res %>% filter(term == "bmi") %>% mutate(term = "bmi (immediate - 90 days)", odd_ratio = exp(estimate), time = 90/365)

for (i in 1:9) {
    obesity_binary <- obesity
    obesity_binary <- add_binary_death_diff_time(obesity_binary,i)

    # Finding the time when a person dies/ no longer participates in the study/ not in the time frame
    obesity_binary <- add_time_to_event(obesity_binary,i)

    # select only columns with age, gender and bmi
    obesity_binary_x_years <- obesity_binary[c(37,38,2:3,8)]

    res <- coxph(Surv(time_to_event_x_years, death_after_x_years) ~ bmi + gender + age, data = obesity_binary_x_years)

    rows <- tidy(res) %>% filter(term == "bmi") %>% mutate(term = paste0(paste0("bmi (",i)," years)"),odd_ratio = exp(estimate), time = i)
    paradox_by_year <-  bind_rows(paradox_by_year,rows)
    }
paradox_by_year 

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
paradox_by_year %>%
  tail(10) %>%
  ggplot( aes(x=time, y=odd_ratio)) +
    geom_line( color="grey") +
    geom_point(shape=21, color="black", fill="#69b3a2", size=5) +  scale_x_continuous(limits = c(0, 9), breaks = seq(0, 9, by = 1)) +  scale_y_continuous(limits = c(0.94, 0.97), breaks = seq(0.94, 0.97, by = 0.01)) +  labs(x = "Time (years)", y="Odds ratio", title="Odds ratio over time from hospitalization")  + theme_classic()  +
        theme(plot.title = element_text(face="bold",size = 16), axis.title.x = element_text(hjust=0.5,size = 14),axis.title.y = element_text(hjust=0.5,size = 14), legend.title = element_text(face="bold", size=13), legend.text = element_text(size=13)) 